In [84]:
import random
import requests
import json
import csv

In [73]:
FILEPATH = 'wikipedia_names.txt'
SEARCH_STRING = 'born'

lines = 0
found = 0

matches = set()

with open(FILEPATH,'r',encoding='utf-8') as f:
    for line in f.readlines():
        parts = line.split(";")
        if SEARCH_STRING in parts[-1]:
            found +=1
            matches.add(line.strip())
        lines +=1

print(f"{SEARCH_STRING} found {found} out of {lines}")
# 25 random lines
matches = list(matches)
random.shuffle(matches)
random_items = matches[0:500]
print(random_items[0:10])

born found 466887 out of 2704010
['[[Hugh Jack]];(born 1929)', '[[Deborah Lurie]];(born 1972)', '[[Gary Yost]];(born 1959)', '[[Maggie Crotty]];(born 1948)', '[[Vasiliy Rats]];(born 1961)', '[[David Bargh]];(born 1962)', '[[Jimmy Algerino]];(born 1971)', '[[Didier Stil]];(born 1964)', '[[Carmine Abbagnale]];(born 1962)', '[[Solbi]];(born 1984)']


In [120]:
# fetch first paragraph of every random person page (with born in data string)
def get_page_via_api(wiki_keyword_raw):
    prep = "_".join(wiki_keyword_raw.split(" "))
    url = f'https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&explaintext=1&titles={prep}&exintro=1'
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            raise ValueError('No response!', response.status_code)
    except Exception as ee:
        print(ee)

x = get_page_via_api('Deborah Lurie')
print(x)
assert 'The Academy of Motion Picture Arts and' in x

{"batchcomplete":"","query":{"normalized":[{"from":"Deborah_Lurie","to":"Deborah Lurie"}],"pages":{"23403146":{"pageid":23403146,"ns":0,"title":"Deborah Lurie","extract":"Deborah Ruth Lurie (born March 9, 1974) is an American composer, arranger, and music producer. Her work has appeared in films such as Dear John, An Unfinished Life, Safe Haven, and 9. She has also been a string arranger for rock and pop performers such as Katy Perry, Kelly Clarkson, and The All-American Rejects. In 2015, she received the ASCAP Shirley Walker Award to honor her achievements contributing to the diversity of film and television music. She was one of the composers interviewed in Score: A Film Music Documentary. The Academy of Motion Picture Arts and Sciences extended membership to her in 2016 for her contributions to motion pictures."}}}}


In [121]:
def get_wiki_extract(json_string):
    r = json.loads(json_string)
    target = r['query']['pages']
    page_key = list(target.keys())[0]
    page_text = target[page_key]['extract']
    return page_text

x2 = get_wiki_extract(x)
print(x2)

Deborah Ruth Lurie (born March 9, 1974) is an American composer, arranger, and music producer. Her work has appeared in films such as Dear John, An Unfinished Life, Safe Haven, and 9. She has also been a string arranger for rock and pop performers such as Katy Perry, Kelly Clarkson, and The All-American Rejects. In 2015, she received the ASCAP Shirley Walker Award to honor her achievements contributing to the diversity of film and television music. She was one of the composers interviewed in Score: A Film Music Documentary. The Academy of Motion Picture Arts and Sciences extended membership to her in 2016 for her contributions to motion pictures.


In [122]:
raw = get_page_via_api('Hugh Jack')

In [123]:
snippet = get_wiki_extract(raw)
print(snippet)

Hugh Reid Jack (19 December 1929 – 19 December 2018) was an Australian long jumper who competed in the 1956 Summer Olympics. Jack competed for Williamstown Athletics Club for 15 years and coached for more than 40 years.


In [146]:
def clean_name(name_string):
    link, date = name_string.split(";")
    link = link.replace('[','').replace(']','').strip()
    date = date.replace('(','').replace(')','').strip()
    return link, date

In [ ]:

#build POSITIVE training set (actual people excerpts)
sucesses = 0
with open('classifier_people.csv','w', encoding='utf-8',newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=['name','extract','class'])
    writer.writeheader()
    for i in matches:
        try:
            link, date = clean_name(i)
            page = get_page_via_api(link)
            if page:
                extract = get_wiki_extract(page)
                if extract:
                    writer.writerow({'name':link, 'extract':extract, 'class':'person'})
                    sucesses +=1
                else:
                    raise ValueError('extraction not found', link)
            else:
                raise ValueError('No page found')
        except Exception as ee:
            print(ee)

print(sucesses)

In [136]:
person_names = set()

with open('classifier_people.csv','r', encoding='utf-8',newline='') as csvfile:
    reader = csv.DictReader(csvfile, fieldnames=['name','extract','class'])
    for row in reader:
        person_names.add(row['name'])

In [137]:
len(person_names)

1651

In [140]:
list(person_names)[0:5]

['Sandip Nandy',
 'Ahmad Alenemeh',
 'Guram Makayev',
 'Adem Zorgane',
 'Béla Mavrák']

In [150]:
#get list of non person names
not_people = set()

with open('wikipedia_names.txt','r',encoding='utf-8', newline='')as f:
    for line in f.readlines():
        if 'born' not in line:
            line, date = clean_name(line)
            decision = input(f'A person:{line}? (y) (n) TALLY: {len(list(not_people))}')
            if not decision.lower()[0] == 'y':
                not_people.add(line)
                print(len(list(not_people)))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200


IndexError: string index out of range

In [158]:
#build NEGATIVE training set (actual people excerpts)
sucesses = 0
with open('classifier_not_people.csv','w', encoding='utf-8',newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=['name','extract','class'])
    writer.writeheader()
    for i in not_people:
        try:
            link = i.replace(" ", "_")
            page = get_page_via_api(link)
            if page:
                extract = get_wiki_extract(page)
                if extract:
                    writer.writerow({'name':link, 'extract':extract, 'class':'person'})
                    sucesses +=1
                else:
                    raise ValueError('extraction not found', link)
            else:
                raise ValueError('No page found')
        except Exception as ee:
            print(ee)

print(sucesses)

('extraction not found', 'Pink_Panther_(character)|Pink_Panther')
('extraction not found', 'CNRS_Gold_medal')
('extraction not found', 'Karl-Ferdinands-Universität|Charles_University_in_Prague')
('extraction not found', 'ISO_646')
'extract'
'extract'
'extract'
('extraction not found', 'ForMemRS')
('extraction not found', '1,000,000,000_(number)|billion')
('extraction not found', 'Model_M')
('extraction not found', 'APG_II_system|APG_II')
'extract'
('extraction not found', 'AWI0005x3s')
('extraction not found', 'Algerian_Revolution')
'extract'
'extract'
('extraction not found', 'Stedelijk_Museum_Amsterdam|Stedelijk_Museum')
'extract'
('extraction not found', 'Society_for_the_Propagation_of_the_Gospel_in_Foreign_Parts')
'extract'
'extract'
'extract'
'extract'
('extraction not found', 'First_World_War')
('extraction not found', 'Córdoba,_Spain|Cordoba')
('extraction not found', 'Andrey_Markov_(Soviet_mathematician)|Andrei_Andreyevich_Markov')
'extract'
('extraction not found', '501(c)')
'

In [156]:
sucesses

0

In [179]:
MAX = 85
people = []
with open('classifier_people.csv','r',encoding='utf-8')as pf1:
    reader = csv.DictReader(pf1, fieldnames=['name','extract','class'])
    for count, row in enumerate(reader):
        if count > 0:
            people.append(row)

not_people = []
with open('classifier_not_people.csv','r',encoding='utf-8')as pf1:
    reader = csv.DictReader(pf1, fieldnames=['name','extract','class'])
    for count, row in enumerate(reader):
        if count > 0:
            not_people.append(row)

In [180]:
people[0:5]

[{'name': 'Hugh Jack',
  'extract': 'Hugh Reid Jack (19 December 1929 – 19 December 2018) was an Australian long jumper who competed in the 1956 Summer Olympics. Jack competed for Williamstown Athletics Club for 15 years and coached for more than 40 years.',
  'class': 'person'},
 {'name': 'Deborah Lurie',
  'extract': 'Deborah Ruth Lurie (born March 9, 1974) is an American composer, arranger, and music producer. Her work has appeared in films such as Dear John, An Unfinished Life, Safe Haven, and 9. She has also been a string arranger for rock and pop performers such as Katy Perry, Kelly Clarkson, and The All-American Rejects. In 2015, she received the ASCAP Shirley Walker Award to honor her achievements contributing to the diversity of film and television music. She was one of the composers interviewed in Score: A Film Music Documentary. The Academy of Motion Picture Arts and Sciences extended membership to her in 2016 for her contributions to motion pictures.',
  'class': 'person'},

In [181]:
not_people[0:5]

[{'name': 'World_Health_Organization',
  'extract': 'The World Health Organization (WHO) is a specialized agency of the United Nations responsible for international public health. The WHO Constitution states its main objective as "the attainment by all peoples of the highest possible level of health". Headquartered in Geneva, Switzerland, it has six regional offices and 150 field offices worldwide.\nThe WHO was established on 7 April 1948. The first meeting of the World Health Assembly (WHA), the agency\'s governing body, took place on 24 July of that year. The WHO incorporated the assets, personnel, and duties of the League of Nations\' Health Organization and the Office International d\'Hygiène Publique, including the International Classification of Diseases (ICD). Its work began in earnest in 1951 after a significant infusion of financial and technical resources.The WHO\'s mandate seeks and includes: working worldwide to promote health, keeping the world safe, and serve the vulnerab

In [184]:
#CREATE BINARY text classification csv
with open('binary_classification_data.csv','w', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['name','extract','class'])
    for i in people:
        writer.writerow(i)
    for i2 in not_people:
        writer.writerow(i2)


In [197]:
from ludwig.api import LudwigModel
import pandas as pd

# train a model
config = {"input_features":[{"name": "extract","type": "text"},{"name":"class","type":"category"}],
          "output_features":[{"name":"class","type":"category"}]}

model = LudwigModel(config)
data = pd.read_csv("D:/WikipediaDUMP/NLP LAB/train_the_model/binary_classification_data.csv")
train_stats, _, model_dir = model.train(data)

# or load a model
model = LudwigModel.load(model_dir)

# obtain predictions
predictions = model.predict(data)

ModuleNotFoundError: No module named 'ludwig'